<a href="https://colab.research.google.com/github/leahandofir/earthformer-inference-experiments/blob/main/earthformer_SEVIR_train_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Trying to run the training code on a small dataset (https://github.com/amazon-science/earth-forecasting-transformer/tree/main/scripts/cuboid_transformer/sevir)

Install dependecies

In [1]:
# install CUDA 11.6.2 https://stackoverflow.com/questions/50560395/how-to-install-cuda-in-google-colab-gpus
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-ubuntu2004.pin
!mv cuda-ubuntu2004.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget https://developer.download.nvidia.com/compute/cuda/11.6.2/local_installers/cuda-repo-ubuntu2004-11-6-local_11.6.2-510.47.03-1_amd64.deb
!dpkg -i cuda-repo-ubuntu2004-11-6-local_11.6.2-510.47.0-1_amd64.deb
!apt-key add /var/cuda-repo-ubuntu2004-11-6-local/7fa2af80.pub
!apt-get update
!apt-get -y install cuda-11-6

--2023-05-14 05:00:42--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-ubuntu2004.pin
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190 [application/octet-stream]
Saving to: ‘cuda-ubuntu2004.pin’

cuda-ubuntu2004.pin 100%[===================>]     190  --.-KB/s    in 0s      

2023-05-14 05:00:42 (2.95 MB/s) - ‘cuda-ubuntu2004.pin’ saved [190/190]

--2023-05-14 05:00:42--  https://developer.download.nvidia.com/compute/cuda/11.6.2/local_installers/cuda-repo-ubuntu2004-11-6-local_11.6.2-510.47.03-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 O

In [2]:
# symlink to be the current CUDA
!rm /usr/local/cuda
!ln -s /usr/local/cuda-11.6 /usr/local/cuda

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_Mar__8_18:18:20_PST_2022
Cuda compilation tools, release 11.6, V11.6.124
Build cuda_11.6.r11.6/compiler.31057947_0


In [4]:
%%shell
# install dependencies
pip install --upgrade pip
python3 -m pip install torch==1.12.1+cu116 torchvision==0.13.1+cu116 -f https://download.pytorch.org/whl/torch_stable.html
python3 -m pip install "pytorch_lightning>=1.6.4,<1.8.0"
python3 -m pip install xarray netcdf4 opencv-python
git clone https://github.com/amazon-science/earth-forecasting-transformer.git
cd earth-forecasting-transformer
python3 -m pip install -U -e . --no-build-isolation
pip install -v --disable-pip-version-check --no-cache-dir pytorch-extension git+https://github.com/NVIDIA/apex.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 GB 465.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 40.5 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu118
    Uninstalling torch-2.0.0+cu118:
      Successfully uninstalled torch-2.0.0+cu118
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.1+cu118
    Uninstalling torchvision-0.15.1+cu118:
      Successfully uninstalled torchvision-0.15.1+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.1+cu118 requires torch=

In [ ]:
# !curl 'https://raw.githubusercontent.com/leahandofir/earthformer-inference-experiments/main/prerequisites.sh' -o ./prerequisites.sh
# !bash ./prerequisites.sh
%env HOME=/content/earth-forecasting-transformer
%cd ~

In [11]:
# download all 2019 data
!aws s3 cp --no-sign-request --recursive --region us-west-2 s3://sevir/data/vil/2019 ~/datasets/sevir/data/vil/2019
!aws s3 cp --no-sign-request --region us-west-2 s3://sevir/CATALOG.csv ~/datasets/sevir/CATALOG.csv

download: s3://sevir/data/vil/2019/SEVIR_VIL_STORMEVENTS_2019_0701_1231.h5 to ../../../datasets/sevir/data/vil/2019/SEVIR_VIL_STORMEVENTS_2019_0701_1231.h5
download: s3://sevir/data/vil/2019/SEVIR_VIL_STORMEVENTS_2019_0101_0630.h5 to ../../../datasets/sevir/data/vil/2019/SEVIR_VIL_STORMEVENTS_2019_0101_0630.h5
download: s3://sevir/data/vil/2019/SEVIR_VIL_RANDOMEVENTS_2019_0901_1231.h5 to ../../../datasets/sevir/data/vil/2019/SEVIR_VIL_RANDOMEVENTS_2019_0901_1231.h5
download: s3://sevir/data/vil/2019/SEVIR_VIL_RANDOMEVENTS_2019_0101_0430.h5 to ../../../datasets/sevir/data/vil/2019/SEVIR_VIL_RANDOMEVENTS_2019_0101_0430.h5
download: s3://sevir/data/vil/2019/SEVIR_VIL_RANDOMEVENTS_2019_0501_0831.h5 to ../../../datasets/sevir/data/vil/2019/SEVIR_VIL_RANDOMEVENTS_2019_0501_0831.h5
download: s3://sevir/CATALOG.csv to ../../../datasets/sevir/CATALOG.csv


In [ ]:
%cd scripts/cuboid_transformer/sevir

In [9]:
from omegaconf import OmegaConf
conf = OmegaConf.load('./cfg_sevir.yaml')
conf.dataset.start_date = [2019, 1, 1]
conf.dataset.end_date = [2019, 12, 1]
conf.dataset.train_val_split_date = [2019, 1, 31]
conf.dataset.train_test_split_date = [2019, 5, 1]
OmegaConf.save(config=conf, f='./my_cfg_sevir.yaml')

In [13]:
!MASTER_ADDR=localhost MASTER_PORT=10001 python train_cuboid_sevir.py --gpus 1 --cfg my_cfg_sevir.yaml --ckpt_name last.ckpt --save tmp_sevir

2023-05-14 04:44:05.824058: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Global seed set to 0
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/content/earth-forecasting-transformer/scripts/cuboid_transformer/sevir/train_cuboid_sevir.py:777: UserWarning: ckpt /content/earth-forecasting-transformer/scripts/cuboid_transformer/sevir/experiments/tmp_sevir/checkpoints/last.ckpt not exists! Start training from epoch 0.
  warnings.warn(f"ckpt {ckpt_path} not exists! Start training from epoch 0.")
Global seed set to 0
Initializing distr